In [4]:
import pandas as pd
import numpy as np
df = pd.read_csv('C:/Users/User/Desktop/Python_Practice/MaintenancePredict/predictive_maintenance.csv')
df = pd.get_dummies(df, columns=['Type'])
df['Air_temperature'] = df['Air temperature [K]']
df['Process_temperature'] = df['Process temperature [K]']
df['Rotational_speed'] = df['Rotational speed [rpm]']
df['Torque'] = df['Torque [Nm]']
df['Tool_wear'] = df['Tool wear [min]']
df = df.drop(['UDI','Product ID','Target','Air temperature [K]','Process temperature [K]','Rotational speed [rpm]','Torque [Nm]','Tool wear [min]'],axis=1)
df.drop(df[df['Failure Type']=='Random Failures'].index, inplace=True)
Failure_Type_map={
    'No Failure':0,
    'Power Failure':1,
    'Tool Wear Failure':2,
    'Overstrain Failure':3,
    'Heat Dissipation Failure':4
}
df['Failure Type']=df['Failure Type'].map(Failure_Type_map)
x = df.iloc[:,1:9]
y = df.loc[:,'Failure Type']

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import learning_curve
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import RandomizedSearchCV

x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=1)
x_resample,y_resample = SMOTE(random_state=1).fit_resample(x_train, y_train)
model = XGBClassifier(n_estimators=800, max_depth=481, subsample=1, colsample_bytree=0.8, learning_rate=0.08, random_state=1)
model.fit(x_resample, y_resample)
y_pred = model.predict(x_test)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      1928
           1       0.76      0.84      0.80        19
           2       0.08      0.12      0.10         8
           3       0.93      0.93      0.93        15
           4       0.93      1.00      0.96        27

    accuracy                           0.98      1997
   macro avg       0.74      0.78      0.76      1997
weighted avg       0.99      0.98      0.99      1997



In [6]:
import pickle
import gzip

with gzip.GzipFile('./AMSModel.pgz', 'w') as f:
    pickle.dump(model, f)